# Some information on this notebook

Google Colab provides an efficient environment for writing codes collaboratively with your **team mates**. 

For us **teachers**, this environment makes it possible to follow your work, whether we are all face-to-face at IMT Atlantique, in hybrid mode, or totally remote. 

This notebook offers you a **skeleton** of an ILS meta-heuristics, pre-configured for the use-case on the hub location problem. 

To collaborate with your team mates on this notebook, you should : 

- First **save a copy of this file in your Google Drive**: 

  Go to **File** / **Fichier**, then click on **Save a copy in Drive** / **Sauver une copie dans Google Drive**. 

- Then, **open that copy**, and **share that copy with your team mates** (and your teacher obviously): 

  Click on the top right on **Share** / **Partager**, then in the window **Obtenir le lien**, change the sharing method **Restricted** / **Limité** to **Anyone with the link** / **Tous les utilisateurs disposant du lien**, and change **Viewer** / **Lecteur** to **Editor** / **Editeur**. Finally click on **Copy link** / **Copier le lien**, and share this link with your team mates. 

The best is probably to put that link in the header of your Mattermost team, so that your teacher and your team mates can easily find it. 

**Before starting to code**, you should execute the first cell below, which helps you to upload the data files (in Excel format) to your notebook. You can also do this by hand, by: 

- Clicking on the left on the small **folder icon**, 
- Creating a **directory** for the data files (call it data), 
- **Uploading** the two Excel files provided in the website (the small and the large problem) in that directory. 


# Upload of the data files (do only once !!)

The cell below contains some code to

- **Upload** the Excel files containing the **data** of the problems,
- **Create** a **directory** for those files (we chose to call that directory `data` for some obvious reasons),
- **Move** the data files to the `data` directory,
- **Remove** the `sample_data` directory which is created by default when a new Google Colab notebook is created.

Once executed at the beginning of your work, you can **comment this cell out** (by putting `#` in front of each line), so that it is not executed every time you execute the whole notebook ! 

In [22]:
import os
import shutil

# uploaded = files.upload()
# dest_path = "/content/data/"
# os.makedirs(os.path.dirname(dest_path), exist_ok=True)
# shutil.move("/content/InputDataHubLargeInstance.xlsx", dest_path)
# shutil.move("/content/InputDataHubSmallInstance.xlsx", dest_path)
# shutil.rmtree("/content/sample_data/", ignore_errors=True, onerror=None)

# Imports of modules

Below we import some modules necessary to make this code work properly. 

You can add other modules here which you might need for your algorithm. 

In [23]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Definitions of constants

In the cell below we define some constants that we need in the meta-heuristic. 

The paths to the data files are also specified here. 

In [24]:
# ILS parameters

ALPHA = 0.05
MAX_ITER = 100     # maximum number of iterations (Stopping criterion of ILS)
MAX_ITER_NI = 50   # number of iterations without improvement of the objective function (Stopping criterion of ILS)
MAX_ITER_LS = 100  # maximum number of iterations of the local search operator (Outer loop)
MAX_SWAPS = 1      # maximum number of swaps of the local search operator (Inner loop)
NB_SWAPS = 1       # number of swaps in the perturbation operator

# Path to data file

INPUT_DATA = "data/InputDataHubSmallInstance.xlsx"  # Small instance
#INPUT_DATA = "/data/InputDataHubLargeInstance.xlsx"  # Large instance

# Functions to load and prepare the data

In the cell below, you find two functions : 

- first, `read_excel_data`, which returns the values of a specific sheet in an Excel file,
- then, `create_data_model` which fills the `data` dictionary with the data from the various sheets of the Excel file, as well as some dependent data, which is calculated from the raw data.

In [25]:
def read_excel_data(filename: str, sheet_name: str) -> np.ndarray:
    """Read data from an excel spreadsheet.

    :param filename:
    :param sheet_name:
    :return: sheet data
    """
    data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
    values = data.values
    return values


def create_data_model() -> dict:
    """Create the data model which gathers all problem parameters.

    :return: data model structure
    """
    data = {}
    # This section contains all required data #
    node_num = read_excel_data(INPUT_DATA, "NodeNum")
    data['node_num'] = node_num[0][0]

    # Flow between Nodes Read From Excel #
    data['flow'] = read_excel_data(INPUT_DATA, "flow(wij)")

    # Cost between Nodes Read from Excel #
    data['var_cost'] = read_excel_data(INPUT_DATA, "varCost(cij)")

    # Fixed Location Cost Read from Excel #
    data['fix_cost'] = read_excel_data(INPUT_DATA, "fixCost(fk)")

    # Discount Factor Read from Excel #
    data['alpha'] = read_excel_data(INPUT_DATA, "alpha")

    # Discount Factor Read from Excel #
    data['cap'] = read_excel_data(INPUT_DATA, "Cap(ckmax)")

    # Dependent Parameters
    data['origin'] = data['flow'].sum(axis=1)
    data['destination'] = data['flow'].sum(axis=0)

    return data

# Functions to calculate the objective function from the solution representation

The cell below contains 2 functions to calculate the objective function of an individual: 
- first `prufer_to_tree` which transforms the Prüfer representation of a solution into a tree,
- second, `tree_to_prufer` which transforms a tree into a Prüfer representation,
- third, `compute_of` which calculates the fitness (or objective function) of an individual (or a solution).

In [26]:
def prufer_to_tree(prufer_sequence: list) -> nx.Graph:
    """Transform the Prüfer representation into a tree.
    
    :param a: Prüfer sequence representing a tree
    :return: tree as a graph structure
    """
    return nx.from_prufer_sequence(prufer_sequence)


def tree_to_prufer(tree: nx.Graph) -> list:
    """Transform a tree into a Prüfer sequence.

    :param tree:
    :return: Prüfer sequence representing the tree
    """
    return nx.to_prufer_sequence(tree)


def compute_of(individual: list, data: dict) -> float:
    """Calculate the objective function of the individual.
    
    :param individual: solution as a tree in Prüfer representation
    :param data: data model including all parameters of the problem
    :return: value of the objective function for given individual

    .. note:: this returns `inf` if the individual doesn't respect the constraints
    """
    fitness = float("inf")

    graph = prufer_to_tree(individual)
    tree_edges = list(graph.edges)
    all_pairs_path = dict(nx.all_pairs_shortest_path(graph))

    hubs = np.unique(individual)
    fixed_cost = data['fix_cost'][hubs].sum()

    variable_cost = 0
    flow_to_hub = np.zeros(data['node_num'])
    for i in range(data['node_num']):
        for j in range(data['node_num']):
            if j > i:
                for x in range(len(all_pairs_path[i][j]) - 1):
                    route = all_pairs_path[i][j]
                    if route[x] in hubs and route[x + 1] in hubs:
                        variable_cost = variable_cost + data['alpha'] * data['var_cost'][route[x], route[x + 1]] * (data['flow'][i, j] + data['flow'][j, i])
                    else:
                        variable_cost = variable_cost + data['var_cost'][route[x], route[x + 1]] * (data['flow'][i, j] + data['flow'][j, i])
    fitness = fixed_cost + variable_cost[0]

    # Calculating the Entering flow to Each Hub #
    for i in range(len(tree_edges)):
        if tree_edges[i][0] not in hubs:
            flow_to_hub[tree_edges[i][1]] = flow_to_hub[tree_edges[i][1]] + (data['origin'][tree_edges[i][0]] + data['destination'][tree_edges[i][0]])
        elif tree_edges[i][1] not in hubs:
            flow_to_hub[tree_edges[i][0]] = flow_to_hub[tree_edges[i][0]] + (data['origin'][tree_edges[i][1]] + data['destination'][tree_edges[i][1]])
    for i in range(len(hubs)):
        flow_to_hub[hubs[i]] = flow_to_hub[hubs[i]] + (data['origin'][hubs[i]] + data['destination'][hubs[i]])

    # Feasibility Check: Capacity Constraint #
    exceed = np.subtract(data['cap'], flow_to_hub)
    exceed_cap = exceed[hubs, hubs]
    if min(exceed_cap) < 0:
        fitness = float("inf")

    return fitness

# Functions to create solutions or individuals

The cell below contains two functions regarding individuals:

- first, `generate_individual` to create a random individual, 
- second, `initial_solution` which returns this single randomly generated individual and its fitness.

In [27]:
def generate_individual(nb_nodes: int) -> list:
    """Generate a random individual.

    The created nodes will be labelled as integer from 0 to `nb_nodes`-1

    :param nb_nodes: total number of nodes in an individual tree
    :return: Prüfer representation of a random tree with given number of nodes
    """
    individual = np.ndarray.tolist(np.random.randint(0, high=nb_nodes-1, size=nb_nodes-2, dtype='int'))
    
    return individual


def initial_solution(data: dict) -> tuple:
    """Generate a random solution and calculate its fitness.
    
    :param data: data model including all parameters of the problem
    :return: random solution in Prüfer representation, and its objective function value
    """
    # here we are generating only one initial solution
    of = float("inf")
    while of == float("inf"):
        solution = generate_individual(data['node_num'])
        of = compute_of(solution, data)
    return solution, of

# Functions for the local search

Below you can find functions to perform a local search: 
- first the general high-level `local_search` function,
- second the `reverse` function, which implements a swap operator, 
- third the `reverse_neighborhood` function which generates the neighborhood based on the reverse operator,
- and finally the `unique_pairs` function, used by `swap_neighborhood`, wich generates unique pairs indexes. 

In [28]:
def local_search(sol: list, of: float, data: dict) -> tuple:
    """Perform a local search.
    
    :param sol: input solution
    :param of: objective function value of input solution
    :param data: data model including all parameters of the problem
    :return: best neighbouring solution, and its objective function value
    """
    # number of iterations local search
    nb_iterations = 0

    best_of = of
    best_sol = sol

    # Main loop local search
    # Local search operators is repeated MAX_ITER_LS times
    while nb_iterations <= MAX_ITER_LS:

        nb_iterations += 1

        # use an operator to perform local search

        # ----> put your code here <----

    return best_sol, best_of


def reverse(p1: int, p2: int, parent: list) -> list:
    """Reverse operator.
    
    This function reverses the nodes between position `p1` and `p2` in the Prüfer sequence.

    :param i: first position in `sol` sequence
    :param j: second position in `sol` sequence
    :param sol: parent solution to modify
    :return: new solution 
    """

    # ----> put your code here <----

    return parent[:p1] + reverse_part + parent[p2:]


def reverse_neighborhood(sol: list, best_of: float, data: dict) -> tuple:
    """Neighborhood generation with a reversion operator.

    All pairs of possible neighbours are investigated and create a solution and its objective function value each.
    
    :param sol: parent solution which neighbourhood to search
    :param best_of: parent solution objective function value
    :param data: data model including all parameters of the problem
    :return: list of neighbouring solutions, and corresponding list of neighbours objective function values
    """

    # ----> put your code here <----

    return n, of


def unique_pairs(n: int):
    """Generator producing pairs of indexes in range(n).
    
    :param n:
    :return: pair of indexes (generator)
    """
    for i in range(n):
        for j in range(i + 1, n):
            yield i, j

# Functions for the perturbation of solutions

In [29]:
def swap(i: int, j: int, sol: list) -> list:
    """Swap operator.
    
    This function simply swaps the nodes at position `i` and `j` in the Prüfer sequence.

    :param i: first position in `sol` sequence
    :param j: second position in `sol` sequence
    :param sol: parent solution to modify
    :return: new solution 
    """

    # ----> put your code here <----

    return sol

def random_swap(sol: list) -> list:
    """Random perturation.
    
    Applies `NB_SWAPS` random swap operations on the solution.

    :param sol: parent solution
    :return: modified solution
    """

    # ----> put your code here <----

    return sol

# This function is the main body of the perturbation operator, wherein the random_swap function is called
def perturb(sol: list, data: dict) -> tuple:
    """Add random perturbations on the solution.

    :param sol: current solution to modify
    :param data: data model including all parameters of the problem
    :return: new solution, and its objective function value
    """

    # ----> put your code here <----

    return pert_sol, pert_of

# Main


In [30]:
if __name__ == "__main__":
    # *************************Initialisation***************************
    # initialise the data
    data = create_data_model()

    # init number of iterations
    nb_iterations = 0

    # find initial solution (just one) with a constructive heuristic
    best_sol, best_of = initial_solution(data)

    # ********************************************************************

    print("Random solution")
    print("Initial objective function value:", best_of)
    print("Solution:", best_sol)

    # **************************Local Search******************************

    best_sol, best_of = local_search(best_sol, best_of, data)
    best_known = best_sol
    best_of_known = best_of

    print("\nLocal Search")
    print("Objective function value:", best_of)
    print("Tour:", best_sol)

    best_solution = prufer_to_tree(best_sol)

    print(best_solution)

    # graph = nx.Graph(best_solution)
    # plt.figure(2)
    # nx.draw(graph, with_labels=True)
    # plt.show()

    # ********************************************************************

    # ******************************ILS***********************************
    flag_continue = True
    improve = 0

    while flag_continue and nb_iterations <= MAX_ITER and improve <= MAX_ITER_NI:

        nb_iterations += 1
        # ******************Perturbation**********************************
        pert_sol, pert_of = perturb(best_sol, data)
        # print(pert_of)

        # ******************Local Search***********************************
        best_sol_pert, best_of_pert = local_search(pert_sol, pert_of, data)
        # print(best_of_pert)

        # ******************Aceptance criterion***************************
        if best_of_pert < best_of_known:
            best_known = best_sol_pert
            best_of_known = best_of_pert
            improve = 0
        else:
            improve += 1

        if best_of_pert < best_of * (1 + ALPHA):
            best_of = best_of_pert
            best_sol = best_sol_pert
        else:
            flag_continue = False

    print("\n")
    print("After", nb_iterations, " ILS iterations, the best solution is:")
    print(best_known)
    print("with total cost:", best_of_known)

    best_solution = prufer_to_tree(best_known)

    graph = nx.Graph(best_solution)
    plt.figure(2)
    nx.draw(graph, with_labels=True)
    plt.show()

Random solution
Initial objective function value: [8596005.15]
Solution: [0, 5, 1, 2, 0, 5]

Local Search
Objective function value: [8596005.15]
Tour: [0, 5, 1, 2, 0, 5]
Graph with 8 nodes and 7 edges


NameError: name 'pert_sol' is not defined